# Mauna Loa CO2 Concentrations

Let's look at monthy-averaged CO2 concentrations measured in Mauna Loa, Hawaii, using data hosted by the NOAA which can be downloaded with
```bash
!wget -q ftp://aftp.cmdl.noaa.gov/products/trends/co2/co2_mm_mlo.txt -O ../data/co2_mm_mlo.txt
```

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from jax import random
import jax.numpy as jnp

import numpyro
import numpyro.distributions as dist
from numpyro import infer
numpyro.set_host_device_count(2) # let's use 2 cores!

import arviz as az